Converting from datetime to seconds: https://stackoverflow.com/questions/40992976/python-convert-datetime-column-into-seconds

See also https://stackoverflow.com/questions/48787670/convert-hours-into-seconds-con-pandas 

and https://datatofish.com/strings-to-datetime-pandas/

CSV to GeoDataFrame: https://anitagraser.com/2019/01/23/from-csv-to-geodataframe-in-two-lines/ 

GeoDataFrame to GeoPackage: https://gis.stackexchange.com/questions/298530/how-do-i-write-a-geopandas-dataframe-into-a-single-file-preferably-json-or-geop


In [1]:
# Imports
import pandas as pd
import geopandas
import datetime
import shapely
import warnings

In [2]:
# Filter future warnings
from shapely.errors import ShapelyDeprecationWarning
#warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Paths
pos_file = '/media/mha114/MassimalDataProcessing/20220324_VideoTransectTest/2022-03-24_144702_Bodo/2022-03-24_144702_Bodo.csv'
save_file = '/media/mha114/MassimalDataProcessing/20220324_VideoTransectTest/test_output.gpkg'

In [4]:
# Read file (only position and timestamp)
data = pd.read_csv(pos_file, usecols=['Lat','Lng','Time'])

In [5]:
# Show dataframe before modifications
data.head()

,Lat,Lng,Time
0,67.289851,14.559417,2022-03-24T13:47:03.196Z
1,67.289851,14.559417,2022-03-24T13:47:03.196Z
2,67.289851,14.559417,2022-03-24T13:47:04.190Z
3,67.289851,14.559417,2022-03-24T13:47:05.216Z
4,67.289851,14.559417,2022-03-24T13:47:06.206Z


In [6]:
# Convert time string to datetime format, and add column with time differences
data['Time'] = pd.to_datetime(data['Time'])
data['TimeDiffSec'] = pd.to_timedelta(data['Time'] - data['Time'][0]).dt.total_seconds()

In [7]:
# Create GeoDataFrame
gdf = geopandas.GeoDataFrame(
    data,
    crs = 'EPSG:4326',
    geometry=geopandas.points_from_xy(data.Lng, data.Lat))

In [8]:
# Show GeoDataFrame
gdf.head()

,Lat,Lng,Time,TimeDiffSec,geometry
0,67.289851,14.559417,2022-03-24 13:47:03.196000+00:00,0.000,POINT (14.55942 67.28985)
1,67.289851,14.559417,2022-03-24 13:47:03.196000+00:00,0.000,POINT (14.55942 67.28985)
2,67.289851,14.559417,2022-03-24 13:47:04.190000+00:00,0.994,POINT (14.55942 67.28985)
3,67.289851,14.559417,2022-03-24 13:47:05.216000+00:00,2.020,POINT (14.55942 67.28985)
4,67.289851,14.559417,2022-03-24 13:47:06.206000+00:00,3.010,POINT (14.55942 67.28985)


In [9]:
# Iterate over rows, add (dummy) filenames
gdf['DummyFile'] = ['DummyFileName_' + str(i).zfill(8) for i in range(gdf.shape[0])]

In [10]:
gdf.head()

,Lat,Lng,Time,TimeDiffSec,geometry,DummyFile
0,67.289851,14.559417,2022-03-24 13:47:03.196000+00:00,0.000,POINT (14.55942 67.28985),DummyFileName_00000000
1,67.289851,14.559417,2022-03-24 13:47:03.196000+00:00,0.000,POINT (14.55942 67.28985),DummyFileName_00000001
2,67.289851,14.559417,2022-03-24 13:47:04.190000+00:00,0.994,POINT (14.55942 67.28985),DummyFileName_00000002
3,67.289851,14.559417,2022-03-24 13:47:05.216000+00:00,2.020,POINT (14.55942 67.28985),DummyFileName_00000003
4,67.289851,14.559417,2022-03-24 13:47:06.206000+00:00,3.010,POINT (14.55942 67.28985),DummyFileName_00000004


In [11]:
# Save as geopackage
gdf.to_file(save_file, driver="GPKG")